# EmoDJ Music Player

EmoDJ is a brand-new AI-powered offline music player desktop application that focuses on improving listeners' emotional wellness.

This application is designed based on psychology theories. It is powered by machine learning to automatically identify music emotion of your songs.

To start EmoDJ at first time, click Cell>Run All<br>
To restart EmoDJ after quit, click Kernel>Restart and Run All

Supported music file format: .wav.
Sample music files in /musics folder are downloaded from Free Music Archive.

### Music Emotion Recognition Engine
Load trained models and predict arousal and valence value of the music <br>

In [1]:
import os
import librosa
import sklearn
        
def preprocess_feature(file_name):
    n_mfcc = 12
    mfcc_all = []
    #MFCC per time period (500ms)
    x, sr = librosa.load(MUSIC_FOLDER + file_name)
    for i in range(0, len(x), int(sr*500/1000)):
        x_cont = x[i:i+int(sr*500/1000)]
        mfccs = librosa.feature.mfcc(x_cont,sr=sr,n_mfcc=n_mfcc)
        #append feature value for music interval shorter than 500ms
        mfccs = np.hstack((mfccs, np.zeros((12,22 - mfccs.shape[1]))))
        mfccs = mfccs.flatten()
        mfcc_all.append(mfccs)
    return np.vstack(mfcc_all)

def normalise(input_data, feature_matrix_mean, feature_matrix_std):
    return (input_data - feature_matrix_mean) / feature_matrix_std

def emotion_predict(file_name):
    #Load trained models
    with open(MODEL_FOLDER + 'arousal_model.pkl', 'rb') as f:
        arousal_model = pickle.load(f)
    with open(MODEL_FOLDER + 'valence_model.pkl', 'rb') as f:
        valence_model = pickle.load(f)
    with open(MODEL_FOLDER + 'feature_matrix_mean.pkl', 'rb') as f:
        feature_matrix_mean = pickle.load(f)
    with open(MODEL_FOLDER + 'feature_matrix_std.pkl', 'rb') as f:
        feature_matrix_std = pickle.load(f)
    mfcc = preprocess_feature(file_name)    
    mfcc_norm = normalise(mfcc, feature_matrix_mean, feature_matrix_std)
    #Predict arousal value per 5 second interval
    music_aro = arousal_model.predict(mfcc_norm).flatten()
    #Predict valence value per 5 second interval
    music_val = valence_model.predict(mfcc_norm).flatten()
    return music_aro, music_val
    

### Music Emotion Retrieval Panel
Display music by their valence and arousal value. Colour of marker represents colour association to average music emotion of that music piece.

Listeners can see annotation (song name, valence value, arousal value) of particular music piece by hovering its marker.

Listeners can retrieve and play the music piece by clicking on its marker.

The music currently playing is shown in yellow colour, the music played is shown in grey colour. This would be reset if the listener select new music piece to play (reconstruct the playlist).

In [2]:
HAPPY_COLOUR = 'lawngreen'
SAD_COLOUR = 'darkblue'
TENSE_COLOUR = 'red'
CALM_COLOUR = 'darkcyan'
BASE_COLOUR = 'darkgrey'
PLAYING_COLOUR = 'gold'
FILE_FORMAT = '.wav'

#Start playing music when user pick the marker on scatter plot
def pick_music(event):
    ind = event.ind
    song_id = emotion_df.iloc[ind , :][ID_FIELD].values[0]
    start_music(song_id)

#Show annotation when user hover the marker on scatter plot
#(song name, valence value, arousal value)
def update_annot(ind):
    pos = scatter_panel.get_offsets()[ind["ind"][0]]
    music_annot.xy = pos
    (x,y) = pos
    song_id = emotion_df[(emotion_df[VAL_FIELD]==x) & (emotion_df[ARO_FIELD]==y)][ID_FIELD].values[0]
    music_annot.set_text(get_song_name(song_id) + \
                        '\nValence: '+ str(x.round(2)) + \
                        '\nArousal: '+ str(y.round(2)))                    

def hover_music(event):
    vis = music_annot.get_visible()
    if event.inaxes == ax_panel:
        cont, ind = scatter_panel.contains(event)
        if cont:
            update_annot(ind)
            music_annot.set_visible(True)
            canvas_panel.draw_idle()
        else:
            if vis:
                music_annot.set_visible(False)
                canvas_panel.draw_idle()

#List marker colour for marks on scatter plot 
#based on corresponding emotion of average arousal valence value of that music
def list_colour_panel(x_list, y_list):
    colour_list = []
    for x, y in zip(x_list,y_list):
        if x >= 0 and y > 0:
            colour_list.append(HAPPY_COLOUR)
        elif x <= 0 and y < 0:
            colour_list.append(SAD_COLOUR)
        elif x < 0 and y >= 0:
            colour_list.append(TENSE_COLOUR)
        else:
            colour_list.append(CALM_COLOUR)
    return colour_list

### Music Visualisation Engine
While playing the music, Fast Fourier Transform was performed on each 1024 frames to show amplitude (converted to dB) and frequency.<br>
Colour of line represents colour association to time vary music emotion.

In [3]:
#Initialise visualition
def init_vis(): 
    line.set_ydata([0] * len(vis_x))
    return line,

#Update the visualisation
#Line plot value based on real FFT (converted to dB)
#Line colour based on emotion of arousal valence value at that time period
def animate_vis(i):  
    global num_CHUNK
    #Show visualisation when
    #-music file is loaded
    #-is playing (not paused)
    #-the music file has not finished playing
    if wf is not None and isplay and wf.getnframes()-CHUNK*num_CHUNK>0:
        num_CHUNK += 1
        data = wf.readframes(CHUNK)
        audio_data = np.frombuffer(data, np.int16)
        dfft = 10.*np.log10(abs(np.fft.rfft(audio_data)+1)) # +1 to avoid log0
        line.set_xdata(np.arange(len(dfft))*10.)
        line.set_ydata(dfft)
        line.set_color(colour_vis.pop(0))
    else:   
        line.set_xdata(vis_x)
        line.set_ydata([0] * len(vis_x))
    return line,

#List colour for marks on scatter plot 
#Based on corresponding emotion of arousal valence value across time period
def list_colour_vis(song_id):
    global colour_vis
    colour_vis = []
    valence_list = valence_df[valence_df[ID_FIELD]==song_id][VAL_FIELD].values[0]
    arousal_list = arousal_df[arousal_df[ID_FIELD]==song_id][ARO_FIELD].values[0]
    for x, y in zip(valence_list,arousal_list):
        if x >= 0 and y > 0:
            colour_vis.extend([HAPPY_COLOUR]*int(TIME_PERIOD*(RATE/CHUNK)))
        elif x <= 0 and y < 0:
            colour_vis.extend([SAD_COLOUR]*int(TIME_PERIOD*(RATE/CHUNK)))
        elif x < 0 and y >= 0:
            colour_vis.extend([TENSE_COLOUR]*int(TIME_PERIOD*(RATE/CHUNK)))
        else:
            colour_vis.extend([CALM_COLOUR]*int(TIME_PERIOD*(RATE/CHUNK)))

colour_vis = []
num_CHUNK = 0
TIME_PERIOD = 5 #5second

### Music Recommendation and Player Engine
In addition to standard functions (such as next, pause, resume), it provides recommended playlist based on similarity of music emotion with the music selection.

It would play the next music piece in playlist automatically, starting from the most similar one, until it reaches the end of playlist.

In [4]:
from tkinter import messagebox
import pygame

def get_song_name(song_id):
    return processed_music[processed_music[ID_FIELD]==song_id][NAME_FIELD].values[0]

def get_song_file_path(song_id):
    return MUSIC_FOLDER +  get_song_name(song_id)

#Construct playlist based on similarity with song selected
#Euclidean distance by valence and arousal value (square root is ignored)
def construct_playlist(song_id):
    global playlist
    playlist = []
    playlist_dict = {}
    curr_val = emotion_df[emotion_df[ID_FIELD]==song_id][VAL_FIELD].values[0]
    curr_aro = emotion_df[emotion_df[ID_FIELD]==song_id][ARO_FIELD].values[0]
    song_list = list(emotion_df[ID_FIELD].values)
    song_list.remove(song_id)
    for compare_song_id in song_list:
        compare_val = emotion_df[emotion_df[ID_FIELD]==compare_song_id][VAL_FIELD].values[0]
        compare_aro = emotion_df[emotion_df[ID_FIELD]==compare_song_id][ARO_FIELD].values[0]
        playlist_dict[compare_song_id] = (curr_val-compare_val)**2 + (curr_aro-compare_aro)**2
    playlist_dict = sorted(playlist_dict.items(), key = lambda kv:(kv[1], kv[0]))
    playlist = [i[0] for i in playlist_dict]

#Update setting to play song 
def update_music_setting(song_id):
    global wf, num_CHUNK
    mixer.music.load(get_song_file_path(song_id))
    mixer.music.play()
    wf = wave.open(get_song_file_path(song_id), 'rb')
    songLabel.set(get_song_name(song_id))
    list_colour_vis(song_id)
    ax_panel.scatter(emotion_df[VAL_FIELD],emotion_df[ARO_FIELD], s=15,c=scatter_colour,picker=False)
    #Played songs are displayed as BASE_COLOUR
    ax_panel.scatter(emotion_df[emotion_df[ID_FIELD].isin(played_songs)][VAL_FIELD], \
                     emotion_df[emotion_df[ID_FIELD].isin(played_songs)][ARO_FIELD], s=16,c=BASE_COLOUR,picker=False)
    #Playing songs are displayed as PLAYING_COLOUR
    ax_panel.scatter(emotion_df[emotion_df[ID_FIELD]==song_id][VAL_FIELD], \
                     emotion_df[emotion_df[ID_FIELD]==song_id][ARO_FIELD], s=17,c=PLAYING_COLOUR,picker=False)
    canvas_panel.draw()
    played_songs.append(song_id)
    num_CHUNK = 0

#User selected in panel to start song and construct new playlist
def start_music(song_id):
    global isplay, played_songs
    mixer.music.stop()
    #Construct playlist
    construct_playlist(song_id)
    played_songs = []
    #Load and play song selected
    isplay = True
    update_music_setting(song_id)

#User clicked next button to play next song in playlist
def next_music():
    global wf, isplay
    mixer.music.stop()
    if playlist:
        isplay = True
        song_id = playlist.pop(0)
        update_music_setting(song_id)
    else:
        wf = None
        isplay = False
        messagebox.showinfo('EmoDJ', 'Reach the end of playlist.')
    
#User clicked pause/resume button  
def pause_music():
    global isplay
    if wf is None and isplay:
        messagebox.showinfo('EmoDJ', 'Please select music in Emotion Panel.')
    elif wf:
        if isplay:
            isplay = False
            mixer.music.pause()
        else:
            isplay = True
            mixer.music.unpause()

#Auto play next music in playlist
def auto_next_music():
    global isplay, wf
    #Check if the song completes
    if isplay:
        if wf is not None:
            if wf.getnframes()-CHUNK*num_CHUNK<=0:
                #End playing if playlist exhaust
                if playlist == [] :
                    isplay = False
                    wf = None
                    songLabel.set('')
                    messagebox.showinfo('EmoDJ', 'Reach the end of playlist.')
                #Play next song in playlist
                elif isplay: 
                    wf = None
                    song_id = playlist.pop(0)
                    update_music_setting(song_id)
    window.after(2000, auto_next_music)
        
wf = None
played_songs = []
playlist = []
isnext = False    
isplay = False

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


### Create Index
Create below search index files
- Average emotion of musics 
- Time varying valence values of musics 
- Time varying arousal values of musics 
- Processed music (Music pieces with music emotion recognised)

In [5]:
import os
import pickle
import time, sys
from IPython.display import clear_output

def update_progress(processed, total):
    bar_length = 20
    progress = processed/total
    
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Music Emotion Recognition Progress: [{0}] {1} / {2} songs processed".format( "#" * block + "-" * (bar_length - block), processed, total )
    print(text)
    
    
def create_index(emotion_df, arousal_df, valence_df, processed_music, music_files):
    #Remove music from processed music if it is not in the folder anymore
    musics_remove = set(processed_music[NAME_FIELD].values) - set(music_files)
    for music_name in musics_remove:
        song_id = processed_music[processed_music[NAME_FIELD]==music_name][ID_FIELD].values[0]
        processed_music = processed_music[processed_music.song_id != song_id]
        emotion_df = emotion_df[emotion_df.song_id != song_id]
        arousal_df = arousal_df[arousal_df.song_id != song_id]
        valence_df = valence_df[valence_df.song_id != song_id]
    
    #Process unprocessed musics in folder
    #Only process .wav files
    musics_new =  set(music_files) - set(processed_music[NAME_FIELD].values)
    num_proceeded = 0
    
    for music_name in musics_new: 
        update_progress(num_proceeded, len(musics_new))

        if music_name.find(FILE_FORMAT)>-1:
            music_aro, music_val = emotion_predict(music_name)
            new_song_id = int(np.nanmax([processed_music[ID_FIELD].max(),0])) +1
            processed_music = processed_music.append({ID_FIELD:new_song_id,NAME_FIELD:music_name}, ignore_index=True)
            arousal_df = arousal_df.append({ID_FIELD:new_song_id,ARO_FIELD:music_aro}, ignore_index=True)
            valence_df = valence_df.append({ID_FIELD:new_song_id,VAL_FIELD:music_val}, ignore_index=True)
            emotion_df = emotion_df.append({ID_FIELD:new_song_id,VAL_FIELD:music_val.mean(),ARO_FIELD:music_aro.mean()}, ignore_index=True)
        num_proceeded += 1
        
    #Save index
    with open(INDEX_FOLDER+'average_emotion.pkl', 'wb') as f:
        pickle.dump(emotion_df,f)
    with open(INDEX_FOLDER+'arousal.pkl', 'wb') as f:
        pickle.dump(arousal_df,f)
    with open(INDEX_FOLDER+'valence.pkl', 'wb') as f:
        pickle.dump(valence_df,f)
    with open(INDEX_FOLDER+'processed_music.pkl', 'wb') as f:
        pickle.dump(processed_music,f)

### Load Index
Load indexes if any. Otherwise, create index folder and empty indexes.<br>
Folder structure:
- musics/ (Music files)
- index/ (Index files)
- model/ (Music emotion recognition model)

In [6]:
import pandas as pd
import numpy as np
MUSIC_FOLDER = 'musics/'
INDEX_FOLDER = 'index/'
MODEL_FOLDER = 'model/'
VAL_FIELD = 'valence'
ARO_FIELD = 'arousal'
NAME_FIELD = 'song_name'
ID_FIELD = 'song_id'

def load_index():
    #For first time using this program
    #Create initial index
    if not os.path.exists(INDEX_FOLDER):
        os.makedirs(INDEX_FOLDER)
    #Average emotion of the music
    try:
        with open(INDEX_FOLDER+'average_emotion.pkl', 'rb') as f:
            emotion_df = pickle.load(f)
    except:
        emotion_df = pd.DataFrame(columns=[ID_FIELD, VAL_FIELD, ARO_FIELD])
    #Dynamic arousal values of the music
    try:
        with open(INDEX_FOLDER+'arousal.pkl', 'rb') as f:
            arousal_df = pickle.load(f)
    except:
        arousal_df = pd.DataFrame(columns=[ID_FIELD, ARO_FIELD])
    #Dynamic valence values of the music
    try:
        with open(INDEX_FOLDER+'valence.pkl','rb') as f:
            valence_df = pickle.load(f)
    except:
        valence_df = pd.DataFrame(columns=[ID_FIELD, VAL_FIELD])
    #Processed music
    try:
        with open(INDEX_FOLDER+'processed_music.pkl','rb') as f:
            processed_music = pickle.load(f)
    except:
        processed_music = pd.DataFrame(columns=[ID_FIELD, NAME_FIELD])
        
    emotion_df = emotion_df.astype({ID_FIELD: int})
    arousal_df = arousal_df.astype({ID_FIELD: int})
    valence_df = valence_df.astype({ID_FIELD: int})
    processed_music = processed_music.astype({ID_FIELD: int})
    
    music_files = os.listdir(MUSIC_FOLDER)
    if '.DS_Store' in music_files:
        music_files.remove('.DS_Store')
    
    return emotion_df, arousal_df, valence_df, processed_music, music_files

### GUI Engine
Graphical user interface to interact with listener.

Before launching GUI, it will check if there are unprocessed music. If so, process to get music emotion values of the unprocessed music and re-create of index.

In [7]:
#Due to system specification difference
#Parameter to ensure synchronisation of visualisation and sound 
SYNC = 3.5 

import tkinter as tk
import wave 
from pygame import mixer
import matplotlib.animation as animation
from matplotlib import style
style.use('ggplot')
import matplotlib.pyplot as plt
import numpy as np
import math
import time
from PIL import ImageTk, Image
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.animation import FuncAnimation

def quit_window():
    mixer.music.stop()
    window.withdraw()
    window.update()
    window.destroy()
    
def cmdNext(): next_music() 
def cmdPause(): pause_music()
def cmdQuit(): quit_window()

emotion_df, arousal_df, valence_df, processed_music, music_files = load_index()

#Check if there are unprocessed music
unprocessed_music = set(music_files).symmetric_difference(set(processed_music[NAME_FIELD].values))
if unprocessed_music:
    print('Processing musics...')
    create_index(emotion_df, arousal_df, valence_df, processed_music, music_files)
    
emotion_df, arousal_df, valence_df, processed_music, _ = load_index()
        
window = tk.Tk()
window.title("EmoDJ")
window.config(background='white')
window.geometry('1300x700')

#EmoDJ logo
logo_image = ImageTk.PhotoImage(Image.open(MODEL_FOLDER + 'logo.jpeg').resize((150, 80), Image.ANTIALIAS))
tk.Label(window, image = logo_image,background='white').grid(row = 0,column=0,sticky='W')

#Song name
songLabel = tk.StringVar()
songLabel.set('')
tk.Label(window, textvariable=songLabel,background='white').grid(row = 1, column=0)

#music visualisation
fig_vis, ax_vis = plt.subplots(figsize=(8,5))
RATE = 44100
CHUNK = 1024
max_x = CHUNK+1
max_y = 100

vis_x = np.arange(0, max_x)
ax_vis.set_xlim(0, max_x)
ax_vis.set_ylim(0, max_y)
ax_vis.set_axis_off()
line, = ax_vis.plot(vis_x, [0] * len(vis_x))
canvas_vis = FigureCanvasTkAgg(fig_vis, master=window)
canvas_vis.get_tk_widget().grid(row=2,column=0,rowspan=2,sticky='W'+'E'+'N'+'S')
ani = animation.FuncAnimation(fig_vis, animate_vis, init_func=init_vis, interval=int(math.ceil(1000/(RATE/CHUNK)))+SYNC, blit=True)

#music player
tk.Button(window, text="Quit", command=cmdQuit).grid(row = 0, column=2,sticky='E'+'N')
mixer.pre_init(frequency=RATE, size=-16, channels=2)
mixer.init()
tk.Button(window, text="Next", command=cmdNext).grid(row = 1, column=1,sticky='W'+'E'+'N'+'S')
tk.Button(window, text="Resume/Pause", command=cmdPause).grid(row = 1, column=2,sticky='W'+'E'+'N'+'S')

#music emotion panel
fig_panel, ax_panel = plt.subplots(figsize=(5,5))
scatter_colour = list_colour_panel(emotion_df[VAL_FIELD], emotion_df[ARO_FIELD])
scatter_panel = ax_panel.scatter(emotion_df[VAL_FIELD],emotion_df[ARO_FIELD], s=15,c=scatter_colour, picker=True)
ax_panel.axvline(x=0,c=BASE_COLOUR)
ax_panel.axhline(y=0,c=BASE_COLOUR)
ax_panel.set_xlim(-1, 1)
ax_panel.set_ylim(-1, 1)
ax_panel.text(-1,0.05,s='-1 Valence',fontsize=9,color=BASE_COLOUR)
ax_panel.text(0.75,0.05,s='+1 Valence',fontsize=9,color=BASE_COLOUR)
ax_panel.text(0.05,-1,s='-1 Arousal',fontsize=9,color=BASE_COLOUR)
ax_panel.text(0.05,1,s='+1 Arousal',fontsize=9,color=BASE_COLOUR)
ax_panel.set_axis_off()
music_annot = ax_panel.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",bbox=dict(boxstyle="round", fc="w"))
music_annot.set_visible(False)
canvas_panel = FigureCanvasTkAgg(fig_panel, master=window)
canvas_panel.get_tk_widget().grid(row=2,column=1,columnspan=2,sticky='W'+'E'+'N'+'S')
canvas_panel.mpl_connect('pick_event', pick_music)
canvas_panel.mpl_connect("motion_notify_event", hover_music)
tk.Label(window, text="Start playing music by clicking on the marker!",background='white').grid(row = 3,column=1,columnspan=2,sticky='W'+'E'+'N'+'S')
window.after(0, auto_next_music)
window.mainloop()
print('Enjoy music! See you next time.')

Enjoy music! See you next time.
